In [1]:
import pandas as pd
import numpy as np
dataset = pd.read_csv('Dataset/Dataset_05.csv')

In [6]:
print(dataset.dtypes)
print(dataset.shape)
print(dataset.columns)
dataset

ID                   int64
Age                  int64
Age_gr               int64
Gender               int64
Work_Experience    float64
Family_Size        float64
Ever_Married         int64
Graduated            int64
Profession           int64
Spending_Score       int64
Var_1                int64
Segmentation        object
dtype: object
(8068, 12)
Index(['ID', 'Age', 'Age_gr', 'Gender', 'Work_Experience', 'Family_Size',
       'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1',
       'Segmentation'],
      dtype='object')


,ID,Age,Age_gr,Gender,Work_Experience,Family_Size,Ever_Married,Graduated,Profession,Spending_Score,Var_1,Segmentation
0,462809,22,30,1,1.0,4.0,1,1,6,2,4,NaN
1,462643,38,40,0,NaN,3.0,2,2,3,0,4,A
2,466315,67,70,0,1.0,1.0,2,2,3,2,6,NaN
3,461735,67,70,1,0.0,2.0,2,2,8,1,6,B
4,462669,40,40,0,NaN,6.0,2,2,4,1,6,A
...,...,...,...,...,...,...,...,...,...,...,...,...
8063,464018,22,30,1,0.0,7.0,1,1,0,2,1,D
8064,464685,35,40,1,3.0,4.0,1,1,5,2,4,D
8065,465406,33,40,0,1.0,1.0,1,2,6,2,6,D
8066,467299,27,30,0,1.0,4.0,1,2,6,2,6,B


In [3]:
# =============================================================================
# 1.위의 표에 표시된 데이터 타입에 맞도록 전처리를 수행하였을 때, 데이터 파일 내에
# 존재하는 결측값은 모두 몇 개인가? 숫자형 데이터와 문자열 데이터의 결측값을
# 모두 더하여 답하시오.
# (String 타입 변수의 경우 White Space(Blank)를 결측으로 처리한다) (답안 예시) 123
# =============================================================================

In [7]:
q1 = dataset.copy()

In [13]:
q1.isnull().sum().sum()

1166

---

In [ ]:
# =============================================================================
# 2.이어지는 분석을 위해 결측값을 모두 삭제한다. 그리고, 성별이 세분화(Segmentation)에
# 영향을 미치는지 독립성 검정을 수행한다. 수행 결과, p-value를 반올림하여 소수점
# 넷째 자리까지 쓰고, 귀무가설을 기각하면 Y로, 기각할 수 없으면 N으로 기술하시오. 
# (답안 예시) 0.2345, N
# =============================================================================

In [15]:
q2 = dataset.copy().dropna()

In [22]:
# 카이스퀘어 검점
# 1. 빈도표 작성
# 2. 카이스퀘어 검정

# 1
q2_tab = pd.crosstab(index=q2.Gender, columns=q2.Segmentation)

# 2
import scipy.stats as sc

q2_out = sc.chi2_contingency(q2_tab)
q2_out[1].round(4) # => pvalue

# pvalue가 유의수준 0.05보다 작으므로 귀무가설을 기각함(Y)
# 정답 : 0.0031 , Y

0.0031

---

In [23]:
# =============================================================================
# 3.Segmentation 값이 A 또는 D인 데이터만 사용하여 의사결정 나무 기법으로 분류
# 정확도를
# 측정해 본다. 
# - 결측치가 포함된 행은 제거한 후 진행하시오.
# - Train대 Test 7대3으로 데이터를 분리한다. (Seed = 123)
# - Train 데이터를 사용하여 의사결정나무 학습을 수행하고, Test 데이터로 평가를
# 수행한다.
# - 의사결정나무 학습 시, 다음과 같이 설정하시오:
# • Feature: Age_gr, Gender, Work_Experience, Family_Size, 
#             Ever_Married, Graduated, Spending_Score
# • Label : Segmentation
# • Parameter : Gini / Max Depth = 7 / Seed = 123
# 이 때 전체 정확도(Accuracy)를 소수점 셋째 자리 이하는 버리고 소수점 둘째자리까지
# 기술하시오.
# (답안 예시) 0.12
# =============================================================================


In [24]:
q3=q2[q2.Segmentation.isin(['A','D'])]

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [27]:
# 학습데이터 분리 7:3
train, test = train_test_split(q3,
                              test_size=0.3,
                              random_state=123)

In [28]:
train.columns

Index(['ID', 'Age', 'Age_gr', 'Gender', 'Work_Experience', 'Family_Size',
       'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1',
       'Segmentation'],
      dtype='object')

In [29]:
x_var=[ 'Age_gr', 'Gender', 'Work_Experience', 'Family_Size',
       'Ever_Married', 'Graduated', 'Spending_Score']

In [31]:
dt = DecisionTreeClassifier(max_depth=7,random_state=123)

In [33]:
dt.fit(train[x_var], train.Segmentation)

pred = dt.predict(test[x_var])

# 정확도
dt.score(test[x_var], test.Segmentation).round(2)

# 정답 : 0.68

0.68